In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_excel("C:/Users/Bravo/Desktop/gearboost/Bares_T35.xlsx")
df.head()

,Unnamed: 0,BRS.T35_Power,BRS.T35_T_GBX_OIL_1,BRS.T35_T_GBX_OIL_2,BRS.T35_T_GBX_T1_HSS,BRS.T35_T_GBX_T1_IMS,BRS.T35_T_GBX_T3_HSS,BRS.T35_T_GBX_T3_IMS,BRS.T35_T_GEAR_BEAR,BRS.T35_Temp_Ambient,BRS.T35_WIND_DIR,BRS.T35_Wind_speed
0,2024-01-25T00:00:00.000,544.650024,55.150002,58.610001,56.509998,52.779999,61.220001,57.910000,61.220001,5.00,101.000000,6.28
1,2024-01-25T00:10:00.000,420.470001,54.810001,57.990002,56.080002,52.410000,60.650002,57.439999,60.650002,4.09,93.879997,5.80
2,2024-01-25T00:20:00.000,404.429993,54.480000,57.490002,55.650002,52.029999,60.090000,56.959999,60.090000,5.00,103.050003,5.82
3,2024-01-25T00:30:00.000,344.700012,54.139999,56.980000,55.209999,51.660000,59.520000,56.490002,59.520000,5.00,96.570000,5.09
4,2024-01-25T00:40:00.000,157.369995,53.799999,56.470001,54.779999,51.279999,58.950001,56.020000,58.950001,5.00,104.849998,4.67


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57024 entries, 0 to 57023
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            57024 non-null  object 
 1   BRS.T35_Power         56003 non-null  float64
 2   BRS.T35_T_GBX_OIL_1   55991 non-null  float64
 3   BRS.T35_T_GBX_OIL_2   55992 non-null  float64
 4   BRS.T35_T_GBX_T1_HSS  55995 non-null  float64
 5   BRS.T35_T_GBX_T1_IMS  56000 non-null  float64
 6   BRS.T35_T_GBX_T3_HSS  55989 non-null  float64
 7   BRS.T35_T_GBX_T3_IMS  55975 non-null  float64
 8   BRS.T35_T_GEAR_BEAR   55989 non-null  float64
 9   BRS.T35_Temp_Ambient  56003 non-null  float64
 10  BRS.T35_WIND_DIR      56600 non-null  float64
 11  BRS.T35_Wind_speed    56003 non-null  float64
dtypes: float64(11), object(1)
memory usage: 5.2+ MB


In [4]:
# 0) Zamanı ayarla
df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'], errors='coerce')
df = df.dropna(subset=['Unnamed: 0']).sort_values('Unnamed: 0').set_index('Unnamed: 0')
df.index.name = 'timestamp'

# 1) 10 dakikalık düzenli ızgaraya oturt
df_rs = df.resample('10T').median()

# 2) Doldurmadan önce eksik maskesi (rapor/özellik için işine yarar)
mask_before = df_rs.isna()

# 3) KISA boşlukları zaman tabanlı interpolate ile doldur
#    limit=6 => en fazla 6 adım (6*10dk=60dk) doldur; daha uzun kesintiler NaN kalsın
df_filled = df_rs.interpolate(method='time', limit=6, limit_direction='both')

# (Opsiyonel) Hangi satırlar dolduruldu? (modele verilebilecek faydalı bir bayrak)
row_interpolated = (mask_before & df_filled.notna()).any(axis=1).astype('int8')
df_filled['row_interpolated'] = row_interpolated

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 57024 entries, 2024-01-25 00:00:00 to 2025-02-23 23:50:00
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   BRS.T35_Power         56003 non-null  float64
 1   BRS.T35_T_GBX_OIL_1   55991 non-null  float64
 2   BRS.T35_T_GBX_OIL_2   55992 non-null  float64
 3   BRS.T35_T_GBX_T1_HSS  55995 non-null  float64
 4   BRS.T35_T_GBX_T1_IMS  56000 non-null  float64
 5   BRS.T35_T_GBX_T3_HSS  55989 non-null  float64
 6   BRS.T35_T_GBX_T3_IMS  55975 non-null  float64
 7   BRS.T35_T_GEAR_BEAR   55989 non-null  float64
 8   BRS.T35_Temp_Ambient  56003 non-null  float64
 9   BRS.T35_WIND_DIR      56600 non-null  float64
 10  BRS.T35_Wind_speed    56003 non-null  float64
dtypes: float64(11)
memory usage: 5.2 MB


In [6]:
df.sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of                      BRS.T35_Power  BRS.T35_T_GBX_OIL_1  BRS.T35_T_GBX_OIL_2  \
timestamp                                                                      
2024-01-25 00:00:00     544.650024            55.150002            58.610001   
2024-01-25 00:10:00     420.470001            54.810001            57.990002   
2024-01-25 00:20:00     404.429993            54.480000            57.490002   
2024-01-25 00:30:00     344.700012            54.139999            56.980000   
2024-01-25 00:40:00     157.369995            53.799999            56.470001   
...                            ...                  ...                  ...   
2025-02-23 23:10:00     411.359985            50.240002            53.209999   
2025-02-23 23:20:00     425.390015            50.139999            52.959999   
2025-02-23 23:30:00     416.339996            50.040001            52.720001   
2025-02-23 23:40:00     369.929993            49.939999   

In [7]:
df.head()

,BRS.T35_Power,BRS.T35_T_GBX_OIL_1,BRS.T35_T_GBX_OIL_2,BRS.T35_T_GBX_T1_HSS,BRS.T35_T_GBX_T1_IMS,BRS.T35_T_GBX_T3_HSS,BRS.T35_T_GBX_T3_IMS,BRS.T35_T_GEAR_BEAR,BRS.T35_Temp_Ambient,BRS.T35_WIND_DIR,BRS.T35_Wind_speed
timestamp,,,,,,,,,,,
2024-01-25 00:00:00,544.650024,55.150002,58.610001,56.509998,52.779999,61.220001,57.910000,61.220001,5.00,101.000000,6.28
2024-01-25 00:10:00,420.470001,54.810001,57.990002,56.080002,52.410000,60.650002,57.439999,60.650002,4.09,93.879997,5.80
2024-01-25 00:20:00,404.429993,54.480000,57.490002,55.650002,52.029999,60.090000,56.959999,60.090000,5.00,103.050003,5.82
2024-01-25 00:30:00,344.700012,54.139999,56.980000,55.209999,51.660000,59.520000,56.490002,59.520000,5.00,96.570000,5.09
2024-01-25 00:40:00,157.369995,53.799999,56.470001,54.779999,51.279999,58.950001,56.020000,58.950001,5.00,104.849998,4.67


In [8]:
# === Vars you can tweak ===
lags = [1, 3, 5]                  # steps
roll_windows = [3, 5, 10]         # steps
add_diff = True
add_pct_change = True
target_col = None                 # e.g., 'failure' if you have a label; else keep None

# --- Select numeric feature columns (exclude target if given)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if target_col and target_col in num_cols:
    num_cols.remove(target_col)

# Safety: if no numeric columns, stop early
assert len(num_cols) > 0, "No numeric feature columns found."

# Work on a copy to avoid chained assignment warnings
_out = df.copy()

# --- LAG FEATURES ---
for lag in lags:
    _out[[f"{c}__lag{lag}" for c in num_cols]] = _out[num_cols].shift(lag)

# --- ROLLING STATS (computed on past values only to avoid leakage) ---
for w in roll_windows:
    past = _out[num_cols].shift(1)  # ensure only history up to t-1 is used
    _out[[f"{c}__r{w}_mean" for c in num_cols]] = past.rolling(w, min_periods=w).mean()
    _out[[f"{c}__r{w}_std"  for c in num_cols]] = past.rolling(w, min_periods=w).std(ddof=0)
    _out[[f"{c}__r{w}_min"  for c in num_cols]] = past.rolling(w, min_periods=w).min()
    _out[[f"{c}__r{w}_max"  for c in num_cols]] = past.rolling(w, min_periods=w).max()

# --- DIFF / PCT-CHANGE ---
if add_diff:
    _out[[f"{c}__diff1" for c in num_cols]] = _out[num_cols].diff(1)

if add_pct_change:
    pct = _out[num_cols].pct_change(1)
    # Replace inf/-inf from tiny denominators
    pct = pct.replace([np.inf, -np.inf], np.nan)
    _out[[f"{c}__pct1" for c in num_cols]] = pct

# --- Drop initial rows made NaN by windows and any remaining NaNs
max_win = 0
if lags: max_win = max(max_win, max(lags))
if roll_windows: max_win = max(max_win, max(roll_windows))
if add_diff or add_pct_change: max_win = max(max_win, 1)

_out = _out.iloc[max_win:]            # remove startup rows with NaNs from windows
_out = _out.dropna(how="any")         # strict: drop rows with any NaN (optional)

# Result: _out is your feature-enriched dataframe usable by XGBoost/LightGBM/CatBoost
df = _out

In [9]:
df.head()

,BRS.T35_Power,BRS.T35_T_GBX_OIL_1,BRS.T35_T_GBX_OIL_2,BRS.T35_T_GBX_T1_HSS,BRS.T35_T_GBX_T1_IMS,BRS.T35_T_GBX_T3_HSS,BRS.T35_T_GBX_T3_IMS,BRS.T35_T_GEAR_BEAR,BRS.T35_Temp_Ambient,BRS.T35_WIND_DIR,...,BRS.T35_T_GBX_OIL_1__pct1,BRS.T35_T_GBX_OIL_2__pct1,BRS.T35_T_GBX_T1_HSS__pct1,BRS.T35_T_GBX_T1_IMS__pct1,BRS.T35_T_GBX_T3_HSS__pct1,BRS.T35_T_GBX_T3_IMS__pct1,BRS.T35_T_GEAR_BEAR__pct1,BRS.T35_Temp_Ambient__pct1,BRS.T35_WIND_DIR__pct1,BRS.T35_Wind_speed__pct1
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-25 01:40:00,285.890015,51.779999,53.419998,52.180000,49.040001,55.549999,53.169998,55.549999,5.00,104.290001,...,-0.006523,-0.009273,-0.008173,-0.007689,-0.009980,-0.008947,-0.009980,0.000000,0.091699,-0.167910
2024-01-25 01:50:00,175.210007,51.450001,52.910000,51.740002,48.669998,54.980000,52.700001,54.980000,5.00,128.199997,...,-0.006373,-0.009547,-0.008432,-0.007545,-0.010261,-0.008840,-0.010261,0.000000,0.229265,0.235426
2024-01-25 02:00:00,181.380005,51.110001,52.400002,51.310001,48.290001,54.410000,52.230000,54.410000,4.28,132.460007,...,-0.006608,-0.009639,-0.008311,-0.007808,-0.010367,-0.008918,-0.010367,-0.144000,0.033229,-0.114338
2024-01-25 02:10:00,52.200001,50.770000,51.669998,49.959999,47.880001,53.580002,51.750000,53.580002,5.00,165.649994,...,-0.006652,-0.013931,-0.026311,-0.008490,-0.015255,-0.009190,-0.015255,0.168224,0.250566,-0.286885
2024-01-25 02:20:00,-21.160000,50.439999,50.139999,46.330002,47.320000,52.080002,51.279999,52.080002,5.00,206.399994,...,-0.006500,-0.029611,-0.072658,-0.011696,-0.027996,-0.009082,-0.027996,0.000000,0.246001,0.295977


In [10]:
# === Basit Anomali Etiketleme (Z-score + çoklu sensör kararı) ===
threshold = 3 # |z| > 3 anomali


# Sayısal kolonları seç
num_cols = df.select_dtypes(include=[np.number]).columns


# Z-score hesapla
z_scores = (df[num_cols] - df[num_cols].mean()) / df[num_cols].std()


# Satır bazlı karar:
# En az iki sensör aynı anda anomaliyse Label=1, aksi halde 0
df["Broken"] = (z_scores.abs() > threshold).sum(axis=1).apply(lambda x: 1 if x >= 2 else 0)

df.head()

,BRS.T35_Power,BRS.T35_T_GBX_OIL_1,BRS.T35_T_GBX_OIL_2,BRS.T35_T_GBX_T1_HSS,BRS.T35_T_GBX_T1_IMS,BRS.T35_T_GBX_T3_HSS,BRS.T35_T_GBX_T3_IMS,BRS.T35_T_GEAR_BEAR,BRS.T35_Temp_Ambient,BRS.T35_WIND_DIR,...,BRS.T35_T_GBX_OIL_2__pct1,BRS.T35_T_GBX_T1_HSS__pct1,BRS.T35_T_GBX_T1_IMS__pct1,BRS.T35_T_GBX_T3_HSS__pct1,BRS.T35_T_GBX_T3_IMS__pct1,BRS.T35_T_GEAR_BEAR__pct1,BRS.T35_Temp_Ambient__pct1,BRS.T35_WIND_DIR__pct1,BRS.T35_Wind_speed__pct1,Broken
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-25 01:40:00,285.890015,51.779999,53.419998,52.180000,49.040001,55.549999,53.169998,55.549999,5.00,104.290001,...,-0.009273,-0.008173,-0.007689,-0.009980,-0.008947,-0.009980,0.000000,0.091699,-0.167910,0
2024-01-25 01:50:00,175.210007,51.450001,52.910000,51.740002,48.669998,54.980000,52.700001,54.980000,5.00,128.199997,...,-0.009547,-0.008432,-0.007545,-0.010261,-0.008840,-0.010261,0.000000,0.229265,0.235426,0
2024-01-25 02:00:00,181.380005,51.110001,52.400002,51.310001,48.290001,54.410000,52.230000,54.410000,4.28,132.460007,...,-0.009639,-0.008311,-0.007808,-0.010367,-0.008918,-0.010367,-0.144000,0.033229,-0.114338,0
2024-01-25 02:10:00,52.200001,50.770000,51.669998,49.959999,47.880001,53.580002,51.750000,53.580002,5.00,165.649994,...,-0.013931,-0.026311,-0.008490,-0.015255,-0.009190,-0.015255,0.168224,0.250566,-0.286885,0
2024-01-25 02:20:00,-21.160000,50.439999,50.139999,46.330002,47.320000,52.080002,51.279999,52.080002,5.00,206.399994,...,-0.029611,-0.072658,-0.011696,-0.027996,-0.009082,-0.027996,0.000000,0.246001,0.295977,0


In [11]:
df["Broken"].value_counts()

0    47450
1     7735
Name: Broken, dtype: int64

In [12]:
df.to_excel("GearBoost Prepared Data.xlsx", index=False)